In [1]:
#Colab
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/aml_final
#with open("github_token.txt", "r") as f:
#  token = f.read()
#! git clone https://{token}@github.com/Tryner/aml_final.git #clone repo
%cd aml_final/
! git pull
! pip install setfit
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\', force_remount=True)\n%cd /content/drive/MyDrive/aml_final\n#with open("github_token.txt", "r") as f:\n#  token = f.read()\n#! git clone https://{token}@github.com/Tryner/aml_final.git #clone repo\n%cd aml_final/\n! git pull\n'

In [2]:
from collections import Counter

import torch
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments
from train.active_learning import ActiveTrainer
from train.active_learning_config import ActiveLearningConfig
from data.dataset_config import DatasetConfig

c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("sst2")
active_learning_config = ActiveLearningConfig()
dataset_config = DatasetConfig()
train_args = TrainingArguments()

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def model_init():
    return SetFitModel.from_pretrained("thenlper/gte-small", use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes}).to(device)
def after_train_callback(trainer: Trainer):
    print("#Accuraccy: " + str(trainer.evaluate()))
def dataset_callback(dataset: Dataset):
    print("#Labels: " + str(Counter(dataset[dataset_config.label_column])))

cpu


In [5]:
trainer = ActiveTrainer(
    model_init=model_init, 
    full_train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    train_args=train_args,
    active_learning_config=active_learning_config, 
    dataset_config=dataset_config,
    after_train_callback=after_train_callback,
    dataset_callback=dataset_callback,
    )

Casting the dataset: 100%|██████████| 8/8 [00:00<00:00, 4000.29 examples/s]


In [6]:
trainer.train()

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Map: 100%|██████████| 8/8 [00:00<00:00, 2668.56 examples/s]
***** Running training *****
  Num examples = 3
  Num epochs = 1
  Total optimization steps = 3
  Total train batch size = 16
  0%|          | 0/3 [00:00<?, ?it/s]

                                          

{'embedding_loss': 0.3072, 'learning_rate': 2e-05, 'epoch': 0.33}



100%|██████████| 3/3 [00:04<00:00,  1.62s/it]
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'train_runtime': 5.2011, 'train_samples_per_second': 9.229, 'train_steps_per_second': 0.577, 'epoch': 1.0}


Epoch:  25%|██▌       | 4/16 [00:29<01:30,  7.58s/it]